In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import mean_squared_error

In [2]:
file_path = "data/EMECS_Raw_Data.xlsx"
countries = ['Canada', 'China', 'France', 'Germany', 'India', 'Italy', 'Japan', 'Russia', 'United Kingdom', 'United States']
reduced_countries = ['India', 'United State', 'Germany', 'Russia', 'China']

# Models
LINEAR_REGRESSION = 'LinearRegression'
RIDGE_CV = 'RidgeCV'
LASSO_CV = 'LassoCV'
models = [LINEAR_REGRESSION, RIDGE_CV, LASSO_CV]

# feature selection
NO_SELECTION = 'no_selection'
MUTUAL_INFO = 'mutual_info'
CORRELATION = 'correlation'
fs_methods = [NO_SELECTION, MUTUAL_INFO, CORRELATION]

In [3]:
last_train_year = 2021
predict_year = 2022

In [4]:
raw_df = pd.read_excel(file_path, sheet_name="sheet1")
df = raw_df

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [5]:
df['year'] = pd.to_datetime(df['year'], format='%Y')
df.index = df['year']
train_df = df[(df.index <= pd.to_datetime(last_train_year, format='%Y'))] 
test_df = df[(df.index == pd.to_datetime(last_train_year, format='%Y'))] # take last year as test data
combined_df = df[(df.index <= pd.to_datetime(last_train_year, format='%Y'))] 

In [6]:
test_df

,country,year,iso_code,code_year,population,gdp,co2,coal_co2,flaring_co2,gas_co2,methane,oil_co2,electricity_generation_twh
year,,,,,,,,,,,,,
2021-01-01,Canada,2021-01-01,CAN,CAN_2021,38155012,2001490000000,537.174,45.894,17.164,228.711,102.600261,235.891,625.95
2021-01-01,China,2021-01-01,CHN,CHN_2021,1425893504,17820500000000,11336.233,7979.436,4.812,764.102,1495.453849,1564.370,8484.02
2021-01-01,France,2021-01-01,FRA,FRA_2021,64531448,2957880000000,306.776,30.421,1.705,85.261,55.838835,178.827,582.28
2021-01-01,Germany,2021-01-01,DEU,DEU_2021,83408560,4259930000000,678.799,232.334,1.831,179.755,49.403934,242.793,581.35
2021-01-01,India,2021-01-01,IND,IND_2021,1407563904,3150310000000,2674.222,1767.297,2.772,133.192,564.055380,621.928,1713.75
2021-01-01,Italy,2021-01-01,ITA,ITA_2021,59240336,2114360000000,337.230,24.486,1.816,152.110,41.053571,147.625,286.39
2021-01-01,Japan,2021-01-01,JPN,JPN_2021,124612528,5005540000000,1062.129,428.082,0.359,211.048,29.799414,390.600,958.53
2021-01-01,Russia,2021-01-01,RUS,RUS_2021,145102752,1836890000000,1711.993,382.146,60.216,854.064,852.768851,376.578,324.89
2021-01-01,United Kingdom,2021-01-01,GBR,GBR_2021,67281040,3122480000000,347.465,24.413,3.531,159.854,48.884225,152.671,582.28


In [7]:
features_cols = train_df.columns.drop(['year', 'country', 'iso_code', 'code_year', 'co2'])
features_cols

Index(['population', 'gdp', 'coal_co2', 'flaring_co2', 'gas_co2', 'methane',
       'oil_co2', 'electricity_generation_twh'],
      dtype='object')

In [8]:
# draw line chart
# fig, ax = plt.subplots(len(features_cols), 1, figsize=(12, 60))
# for i, feature in enumerate(features_cols):
#   for country in countries:
#     df[df['country'] == country].plot.line(ax=ax[i], y=feature, title=feature, label=country)

## Linear Regression

In [9]:
def add_result_row(country, model_name, fs_method, actual_co2, predicted_co2, mse, err_percent, alpha, features):
  return {'Country': country, 
          'Model': model_name, 
          'MSE': mse, 
          'Error Percentage': err_percent,
          'Feature Selection': fs_method,
          'Type': 'Countrywise',
          'Features': features,
          f'Forecast Year ({predict_year}) Actual': actual_co2, 
          f'Forecast Year ({predict_year}) Predicted': predicted_co2, 
          'Alpha': alpha}

def calculate_error_percentage(predicted, actual):
  return (abs(predicted - actual)/ actual) * 100

def calculate_mse(actual, predicted):
  return mean_squared_error(actual, predicted)

### Correlation Feature Selection

In [10]:
def get_high_correlated_features(train_df):
  # Step1: check which features are high correlated with co2
  corr_columns = train_df.columns.drop(['year', 'country', 'iso_code', 'code_year']).tolist()
  # corr_columns = features_cols.tolist()+['co2']
  cor = train_df[corr_columns].corr()
  cor_target = abs(cor["co2"])
  relevant_features = cor_target[cor_target > 0.5]
  # print(f"relevant-to-co2 features:\n{relevant_features.index}") # These features are highly correlated with co2 value

  # Step2: check if remaining features have high correlated tendency with each other
  related_to_co2_features = relevant_features.index.drop('co2')
  relevant_features_arr = train_df[related_to_co2_features].to_numpy()
  correlation_matrix = np.corrcoef(relevant_features_arr, rowvar=False)
  corr_matrix_df = pd.DataFrame(correlation_matrix, columns=related_to_co2_features, index=related_to_co2_features)
  corr_matrix_df = corr_matrix_df[corr_matrix_df <= 0.95] # These features are suitable for training
  selected_features = corr_matrix_df.index
  # print(f"correlation matrix:\n{selected_features}")
  return selected_features

### Mutual Information

In [11]:
def select_k_best_mutual_info(train_df, test_df):
  X_train = train_df[features_cols]
  # X_test = test_df[features_cols]
  y_train = train_df['co2']
  y_test = test_df['co2']

  # First: not choosting any feature

  # Evaluate the performance of the model with different number of features
  mses = []

  for k in range(1, len(features_cols)+1):
    fs = SelectKBest(score_func=mutual_info_regression, k=k).fit(X_train, y_train)
    selected_features = fs.get_feature_names_out(features_cols)
    
    X_train_selected = train_df[selected_features]
    X_test_selected = test_df[selected_features]

    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)

    mses.append(calculate_mse(y_test, y_pred))

  # Determine the optimal number of features
  optimal_k = mses.index(min(mses)) + 1
  # print("mses: ", mses)
  # print(f"Optimal number of features: {optimal_k}")
  return optimal_k

In [12]:
# select_k_best_mutual_info(train_df, test_df)

In [13]:
def get_features_by_mutual_info(train_df, test_df) -> list:
  k_value = select_k_best_mutual_info(train_df, test_df)
  X_train = train_df[features_cols]
  y_train = train_df['co2']
  # print("k_value: ", k_value)
  fs = SelectKBest(score_func=mutual_info_regression, k=k_value).fit(X_train, y_train)
  return fs.get_feature_names_out(features_cols)

### Linear / Regulation - Ridge CV , Lasso CV

In [14]:
def linearRegression(X_train, y_train, X_test, y_test):
  model = LinearRegression()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  mse = calculate_mse(y_test, y_pred)
  # print(f'MSE: {mse}')
  return model, mse, None

In [15]:
def ridgeCV(X_train, y_train, X_test, y_test):
  alphas = np.arange(0.0001, 1000, 0.0005)
  ridge_cv_model = RidgeCV(alphas=alphas)
  ridge_cv_model.fit(X_train, y_train)
  y_pred = ridge_cv_model.predict(X_test)
  mse = calculate_mse(y_test, y_pred)
  # print(f'MSE: {mse}\nalpha: {ridge_cv_model.alpha_}')
  return ridge_cv_model, mse, ridge_cv_model.alpha_

In [16]:
def lassoCV(X_train, y_train, X_test, y_test):
  alphas = np.arange(0.0001, 20, 0.0005)
  lasso_cv_model = LassoCV(alphas=alphas, tol=0.1, max_iter=500000)
  lasso_cv_model.fit(X_train, y_train)
  y_pred = lasso_cv_model.predict(X_test)
  mse = calculate_mse(y_test, y_pred)
  # print(f'MSE: {mse}\nalpha: {lasso_cv_model.alpha_}')
  return lasso_cv_model, mse, lasso_cv_model.alpha_

In [17]:
model_dict = {LINEAR_REGRESSION: linearRegression, RIDGE_CV: ridgeCV, LASSO_CV: lassoCV}
def process_lr(all_train_df, all_test_df, model_name, fs_method):
  result = []
  for country in countries:
    train_df = all_train_df[(all_train_df['country'] == country)]
    test_df = all_test_df[(all_test_df['country'] == country)]
    # Feature Selection
    selected_features = features_cols
    if fs_method == CORRELATION:
      selected_features = get_high_correlated_features(train_df)
    elif fs_method == MUTUAL_INFO:
      selected_features = get_features_by_mutual_info(train_df, test_df)

    # Set X, y train and test data
    X_train = train_df[selected_features]
    X_test = test_df[selected_features]
    y_train = train_df['co2']
    y_test = test_df['co2']

    params = {'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test}
    model, mse, alpha = model_dict[model_name](**params)

    combined_df_country = combined_df[(combined_df['country'] == country)]
    predicted_co2 = model.predict(combined_df_country[selected_features])
    # print(predicted_co2)
    actual_co2 = combined_df_country['co2'].values[0]
    err_percent = calculate_error_percentage(actual_co2, predicted_co2[0])
    features_txt = '\n'.join(selected_features)
    if len(selected_features) == 8:
      features_txt = 'all'
    result.append(add_result_row(country, model_name, fs_method, actual_co2, predicted_co2[0], mse, err_percent, alpha, features_txt))
  return result

In [18]:
final_result = []
for model_name in models:
  for fs_method in fs_methods:
    final_result += process_lr(train_df, test_df, model_name, fs_method)

In [19]:
all_result_df = pd.DataFrame.from_dict(final_result)
all_result_df.to_csv('result_lr_by_country.csv', index=False)